<img src='data/images/section-notebook-header.png' />

# Text Classification - Basics

Text classification in NLP refers to the task of automatically categorizing or assigning predefined labels or categories to textual data based on its content. It is a fundamental problem in NLP and has various applications, including sentiment analysis, spam detection, topic categorization, intent recognition, language identification, and more.

The goal of text classification is to train a model that can learn patterns and features from labeled training data and use that knowledge to classify new, unseen text into predefined categories. These categories can be binary (e.g., positive/negative sentiment) or multiclass (e.g., sports, politics, entertainment).

The process of text classification typically involves the following steps:

* **Data Preparation:** This step involves preprocessing the textual data, which may include tasks like removing punctuation, converting text to lowercase, tokenization (splitting text into individual words or subwords), and removing stop words (commonly occurring words that do not carry significant meaning).

* **Feature Extraction:** In order to train a classification model, textual data needs to be transformed into numerical features that can be understood by machine learning algorithms. Common techniques for feature extraction include bag-of-words representation, word embeddings (such as Word2Vec or GloVe), and more advanced methods like BERT or Transformer-based models.

* **Model Training:** After extracting features, a classification model is trained using a labeled dataset. Popular algorithms for text classification include Naive Bayes, Support Vector Machines (SVM), logistic regression, decision trees, random forests, and neural network architectures such as recurrent neural networks (RNNs) or convolutional neural networks (CNNs).

* **Model Evaluation:** The trained model is evaluated using evaluation metrics such as accuracy, precision, recall, F1 score, or area under the receiver operating characteristic curve (AUC-ROC) to measure its performance. The model's performance on a separate test set or through cross-validation provides an estimate of its generalization ability.

* **Prediction/Inference:** Once the model is trained and evaluated, it can be used to classify new, unseen text by predicting the appropriate category or label based on the learned patterns and features.

Text classification is an active research area in NLP, and various techniques and approaches have been developed to improve the accuracy and efficiency of classification models. These advancements include deep learning models, transfer learning, pretraining on large-scale language models, and leveraging contextual embeddings for better representation of text.

In this notebook, we go through all the steps outlined above using a simple binary datasets to illustrate the task of text classification on the most basic level.

## Setting up the Notebook

### Import all Required Packages

In [ ]:
import numpy as np
import pandas as pd
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import KFold, cross_val_score

from datetime import datetime 

from src.nlputil import preprocess_text

---

## Load & Prepare Dataset

The [sentence polarity dataset](https://www.kaggle.com/datasets/nltkdata/sentence-polarity) is a well-known dataset commonly used for sentiment analysis and text classification tasks in NLP. It consists of sentences or short texts labeled with their corresponding sentiment polarity (positive or negative). This dataset is often used to train and evaluate models that aim to classify text into positive or negative sentiment categories. It serves as a benchmark for sentiment analysis tasks and provides a standardized dataset for researchers and practitioners to compare and evaluate the performance of different algorithms and techniques.

There are several versions and variations of the sentence polarity dataset available, created for different purposes and domains. One of the popular versions is the Movie Review Dataset, also known as the Pang and Lee dataset, created by Bo Pang and Lillian Lee. This dataset contains movie reviews from the website IMDb, with each review labeled as positive or negative. The sentence polarity dataset enables researchers and developers to build and test sentiment analysis models that can automatically determine the sentiment expressed in text, allowing applications such as sentiment monitoring, opinion mining, and customer feedback analysis.

The dataset consists of 2 text files, one containing 5,331 positive sentences and the other 5,331 negative sentences.


### Read Data Files

We use `pandas` to read both files, giving us 2 `pandas` dataframes containing all positive and negative sentences respectively.


In [ ]:
df_sent_pos = pd.read_csv('data/corpora/sentence-polarity-dataset/sentence-polarity.pos', sep='\t', header=None)
df_sent_neg = pd.read_csv('data/corpora/sentence-polarity-dataset/sentence-polarity.neg', sep='\t', header=None)

df_sent_pos.head()

### Create internal representation of dataset

For the training and testing, we want 2 lists, one containing all sentences and another containing the respective labels (here `0` representing negative and `1` representing positive sentences). This split into these 2 lists is very common and the expected input for most off-the-shell implementations of various classification algorithms. Note that there is nothing special about labeling the classes. We could equally use the strings `"negative"` and `"positive"`. Some additional explanations for the code cell below:

- The list method `A.extend(B)` attaches list `B` to list `A`

- `[0]*len(df_sent_neg)` creates a a list `[0, 0, 0, 0, 0, ...]` of length $N$ with $N$ being the number of, here, negative sentences

- `np.array(A)` converts a normal n-dimensional Python list into an n-dimensional numpy array (see `import numpy as np` above). It is not crucial since methods for training and test take both standard lists and numpy arrays as input, but numpy arrays come with a long list of useful functions and features.


In [ ]:
# Create a list for all sentences and ad the sentences from both read files
sentences = []
sentences.extend(df_sent_neg[0].tolist())
sentences.extend(df_sent_pos[0].tolist())

# Preprocess sentences (by default, we only lowercase all letter and remove topwords and punctuation)
sentences_preprocessed = [''] * len(sentences)
for idx, sent in enumerate(sentences):
    sentences_preprocessed[idx] = preprocess_text(sent)

# Create a list for all lables
polarities = []
polarities.extend([0]*len(df_sent_neg))
polarities.extend([1]*len(df_sent_pos))

# Convert from lists to numpy arrays
sentences = np.array(sentences_preprocessed)
polarities = np.array(polarities)

Right now, the dataset contains 5,331 positive sentences followed by 5,331 negative sentences. Before we can split the dataset into training and test data, we first have to shuffle the order to ensure a balanced dataset to in turn ensure a balanced training and test data size. Some additional explanations for the code cell below:

- `combined = list(zip(sentences, polarities))`: We have 2 lists containing the sentences and the labels. Of course, we have to ensure that both lists are shuffled the same way so that each label keeps associated with the same sentence. The `zip()` method accomplishes this, both zipping and unzipping.

- `random.seed(int)` (optional): the `shuffle()` method does not truly randomize the order of the elements of a list, but generates a "pseudo-randomized" order. This in turn allows that, by providing a fixed $seed$, we can ensure that shuffling always returns the same "random" order. This makes the whole process deterministic and can be useful to find problems.


In [ ]:
combined = list(zip(sentences, polarities))

random.seed(1) # (optional)
random.shuffle(combined)

# split the "zipped" list into the two lists of sentences and labels/polarities
sentences[:], polarities[:] = zip(*combined)

### Generate training and test data

Given 100% of the data, a common way is to split it into training data and test data. The training is only done using the training and the test only using the testing data, respectively. To make meaningful statements about the effectiveness of the classifier requires (at least) that the testing is done using data the classifier has never seen before. Some additional explanations for the code cell below:

- `A[:n]` returns the first $n$ elements of list A

- `A[n:]` returns all the elements after the $n$-th elements of list A


In [ ]:
# Let's go for a 80%/20% split -- you can change the value anf see its effects
train_test_ratio = 0.8

# Calculate the size of the training data (the size of the dest data is also implicitly given)
train_set_size = int(train_test_ratio * len(sentences))

# Split data and labels into training and test data with respect to the size of the test data
X_train, X_test = sentences[:train_set_size], sentences[train_set_size:]
y_train, y_test = polarities[:train_set_size], polarities[train_set_size:]

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))

---

## Generate Features Based Using the Vector Space Model (VSM)

The feature set is the document n-gram matrix with n-grams of size 1 and 3. You can change this value, e.g., `ngram_range=(1, 1)` to consider only unigrams (i.e, individual terms/tokens) or `ngram_range=(1, 2)` to consider only unigrams and bigrams. Much larger values are less common since the size of the feature vectors quickly explodes.

First, we define the `TfidfVectorizer` give out specification...


In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

...and then generate the feature set as the TF-IDF term-document matrix.

In [ ]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

num_samples, num_features = X_train_tfidf.shape
print("#Samples: {}, #Features: {}".format(num_samples, num_samples))

---

## Train classifier

We use the `LogisticRegression` classifier from `sklearn`. Just note that we specify `solver="sag"` as the default solver (`"lbfgs"`) would yield a memory error (Jupyter notebooks are by default memory constraint).

**Important:** The `LogisticRegression` classifier from `sklearn` does not implement Gradient Descent like presented in the lectures but implements more sophisticated methods making use of the fact that the loss function for Logistic Regression is convex. This allows for techniques that are much more performant than Gradient Descent but on the other hand not as flexible.


In [ ]:
logistic_regression_classifier = LogisticRegression(solver="sag").fit(X_train_tfidf, y_train)

Using packages such as `sklearn` the basic training of a classifier often comes down to a single line of code. In practice, you typically need to extend this code in case of cross validation and hyperparameter tuning -- see the examples further down below -- but even then existing packages will help a lot to make it simple and keep your code clean.

---

## Evaluate Classifier

Of course, we now want to know how well our trained classifier will perform on the test data. For this, we first need to generate the TF-IDF term-document matrix for the test data given the vocabulary of the vectorizer derived from the training data by using the method `transform()` instead of `fit_transform()`.


In [ ]:
X_test_tfidf = tfidf_vectorizer.transform(X_test)

We  can now use the trained classifier to predict the polarities for the test data.

In [ ]:
y_pred = logistic_regression_classifier.predict(X_test_tfidf)

The method `classification_report()` is a very quick way to show the results of the evaluation by means of precision, recall, and f1-score for each class (here only 2) as well as the average precision, recall, and f1-score. By default, the average is a weighted average. The weight is the support, i.e., the number of data items for each class.

In [ ]:
print(classification_report(y_test, y_pred))

With `ngram_range=(1, 1)` for the vectorizer, you should see an average f1 score of around 0.78. To get a first idea if this is a good result or not, consider that a random guesser would be 50% correct all the time, yielding an f1 score of about 0.5. Thus, we can say that our trained classifier performs better than a random guesser but arguably not that much.

---

## Training Using K-Fold Cross Validation

**K-fold cross-validation** is a technique used in machine learning and model evaluation to assess the performance and generalization ability of a model on a given dataset. It helps to provide a more reliable estimate of a model's performance by reducing the variance associated with a single train-test split.

The k-fold cross-validation process involves the following steps:

* **Data Splitting:** The original dataset is randomly divided into k equal-sized subsets or folds. Each fold is typically a contiguous subset of the data.

* **Model Training and Evaluation:** The model is trained and evaluated k times. In each iteration, one of the k folds is used as the validation set, and the remaining k-1 folds are used as the training set. The model is trained on the training set and evaluated on the validation set. This process is repeated for each fold, ensuring that each fold serves as the validation set exactly once.

* **Performance Aggregation:** The performance results obtained from each fold are collected and aggregated to compute an overall performance metric. Common performance metrics include accuracy, precision, recall, F1 score, or mean squared error, depending on the nature of the problem being addressed.

* **Performance Estimation:** The aggregated performance metric provides an estimate of the model's performance on unseen data, which is often considered a better representation of its true performance compared to a single train-test split. This estimation helps in assessing the model's generalization ability and can be used for model selection or hyperparameter tuning.

The choice of the value k depends on the dataset size and the computational resources available. Common values for k include 5, 10, or even higher values. Smaller values of k may lead to higher variance in performance estimation, while larger values may result in increased computational cost.

K-fold cross-validation helps to address issues like overfitting, where a model performs well on the training data but fails to generalize to unseen data. By evaluating the model on multiple different subsets of the data, it provides a more robust estimate of the model's performance and helps to identify potential issues like overfitting or underfitting.

Given its importance, k-fold cross validation is commonly implemented by packages such as `sklearn`. In the code cell below, we perform 10-fold cross validation (specified by `cv=10`) using a single line of code using the provided method `cross_val_score()`. Note that we use only the training data (`X_train_tfidf` and `y_train`) for the cross validation which is only 80% of the whole dataset. This adheres to the notion to use the training data and validation data for each fold; the test data remains "unseen" during cross validation!

In [ ]:
f1_scores_list = cross_val_score(LogisticRegression(solver="sag"), X_train_tfidf, y_train, cv=10, scoring ='f1')

print(f1_scores_list)

Usually, the reported results are the average scores and the standard variation.

In [ ]:
print("f1 score (mean/average): {:.3f}".format(f1_scores_list.mean()))
print("f1 score (standard deviation): {:.3f}".format(f1_scores_list.std()))

A low standard deviation for the results is a good sign. If some scores differ too much from the rest usually indicates that (a) the dataset is not well shuffled to ensure a reasonably balanced training and validation dataset or that (b) that the size of the dataset is simply not large enough to properly learn in all cases.

---

## A Complete Example with Hyperparameter Tuning

So far we trained and evaluated a classifier by simply deciding on the values of various parameters, here: the sizes of considered n-grams. More specifically, at least with the default value `ngram_range=(1, 1)` we only consider unigrams. However, maybe also including bigrams or trigrams would improve the results. There are also other parameters that we could change beyond their default values. For example, by default, the vectorizer considers all available n-grams (given the specified sizes). But perhaps limiting the number of n-grams to only include the 5,000 most frequent ones but show better results. For this, we could set the `max_features` parameter of our vectorizer. Those parameters that we have to set and which are not learned are called **hyperparameters**. The process of finding the best parameter values is called **hyperparameter tuning** or **hyperparameter optimization**. It involves searching through a predefined space of hyperparameters, typically using techniques such as grid search, random search, or more advanced methods like Bayesian optimization or genetic algorithms.

The process of hyperparameter tuning typically involves the following steps:

* **Define the Hyperparameter Space:** Identify the hyperparameters and define their respective ranges or values that need to be explored.

* **Choose a Search Strategy:** Select an appropriate search strategy, such as grid search or random search, to explore the hyperparameter space. Grid search exhaustively evaluates all possible combinations, while random search randomly samples from the space.

* **Train and Evaluate Models:** For each combination of hyperparameters, train a model using the training data and evaluate its performance on a validation set or using cross-validation. Performance metrics such as accuracy, precision, recall, or F1 score are used to measure the model's performance.

* **Select the Best Hyperparameters:** Based on the performance results, select the combination of hyperparameters that yields the best performance.

* **Test the Final Model:** Once the best hyperparameters are selected, train the model using the entire training dataset and evaluate its performance on a separate test set. This provides an unbiased estimate of the model's performance.

Hyperparameter tuning helps to improve a model's performance by finding the optimal configuration for the given task and dataset. It can significantly impact a model's accuracy, generalization ability, and convergence speed. Effective hyperparameter tuning is essential for building robust and high-performing machine learning models.

### Perform Hyperparameter Tuning

To keep things simple, we perform a very simple example of hyperparameter tuning. On the one hand, we consider only 2 classification algorithms (Logistic Regression and Linear Support Vector Classification) and try 4 different maximum n-gram sizes (up to 4). This gives as 2 x 4 = possible configuration for which we have to train and evaluate a model.

Trying simply all possible combinations is the simplest search strategy. However, consider a hyperparameter with a continuous range (e.g., regularization strength). In this cases, trying *all* possible values are not feasible. Another problem arises if there are simply too many hyperparameters to potentially consider. This requires more sophisticated search strategies beyond just "trying all" in practice. But that's beyond our scope here.

In [ ]:
# We need to keep track of the parameter values that yield the best results
best_score = -1.0
best_classifier = None
best_ngram_size = -1

# Hyperparameter 1: choice of classification model
classifiers = [ LinearSVC(), LogisticRegression(solver="sag") ]

# Hyperparameter 2: max. n-gram sizes
ngram_sizes = [1, 2, 3, 4]

# Trying all possible combinations simply translated to a nester loop iterating over all values for both hyperparameters
for classifier in classifiers:
    for s in ngram_sizes:
        # Save the start time
        start_time = datetime.now() 
        # Define vecotrizer to generate feature set as document ngram matrix
        tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, s))
        # Generate feature set as document ngram matrix
        X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
        # Perform 10-fold valdiation only over training data
        f1_scores_list = cross_val_score(classifier, X_train_tfidf, y_train, cv=10, scoring ='f1')
        # Calculate the average core
        average_f1_score = f1_scores_list.mean()
        # Caluclate the required runtime for this parameter setting:
        time_elapsed = datetime.now() - start_time
        # Print results for current setting
        print("Classifier: {}, n-gram size: {} ==> f1-score: {:.3f} [{}]".format(type(classifier).__name__, s, average_f1_score, time_elapsed))
        # If the average score is better than the current best score, save all the current parameter values
        if average_f1_score > best_score:
            best_score = average_f1_score
            best_ngram_size = s
            best_classifier = classifier

print()
print("Best f1-score: {:.3f} [classifier: {}, n-gram size: {}]".format(best_score, type(best_classifier).__name__, best_ngram_size))

In this basic example, where we don't have many combinations of hyperparameter values, we can easily print and inspect the result for each combination. Some quick take-away messages we can see:

- More features do not automatically yield better results

- More features usually also result in larger training times (which is not surprising)

### Testing with the Best Parameter Values

Having identified the best parameter settings (i.e., which classifier and which n-gram size yield the best result), we can train a classifier over the whole training data -- in contrast to cross validation where we split the training data further into a validation set. Note that this is the one and only time we touch the test data `X_test`.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, best_ngram_size))

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

best_classifier = best_classifier.fit(X_train_tfidf, y_train)
y_pred = best_classifier.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

The final results are a bit better than the ones from the cross validation since the training has been done over the whole training data. These numbers are finally the ones that are usually reported. That the average f1-score and the accuracy is almost identical is not a given, but is the result of an almost perfectly balanced dataset.

---

## Summary

Text classification is a vital task in natural language processing (NLP) that involves automatically assigning predefined labels or categories to textual data based on its content. It has numerous applications, including sentiment analysis, spam detection, topic categorization, intent recognition, and language identification. The process of text classification typically involves data preparation, feature extraction, model training, evaluation, and prediction/inference. Preprocessing steps such as tokenization and removing stop words are performed to transform the text into numerical features. Models such as Naive Bayes, Support Vector Machines, logistic regression, and neural networks are commonly used for text classification.

To train a text classification model, labeled training data is used to learn patterns and features associated with different categories. The model generalizes this knowledge to classify unseen text accurately. Evaluation metrics such as accuracy, precision, recall, and F1 score are employed to assess the model's performance. The field of text classification has seen significant advancements with the introduction of deep learning models, transfer learning, and contextual embeddings, which have improved accuracy and efficiency.

Text classification is widely utilized in industries for tasks like sentiment monitoring, opinion mining, and customer feedback analysis. It allows businesses to understand public sentiment, identify emerging trends, and make informed decisions. Moreover, researchers continue to explore new techniques and datasets to enhance the performance of text classification models, making it an active area of research in NLP. Overall, text classification plays a crucial role in extracting meaningful insights from textual data and has a broad range of practical applications in various domains.